In [6]:
%%capture

import sys

%pip install pandas
%pip install lxml
%pip install matplotlib
%pip install tqdm
%pip install spacy
%pip install wordcloud
!{sys.executable} -m spacy download fr_core_news_md


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import spacy
from tqdm import tqdm
from wordcloud import WordCloud

In [5]:
nlp = spacy.load('fr_core_news_md')

In [ ]:
DATAFILE = "data/train.xml"

df : pd.DataFrame = pd.read_xml(DATAFILE)
df

In [ ]:
df.nunique()


In [ ]:
s = df['note'].value_counts()
s = s.sort_index()
fig = plt.figure()
s.plot.bar()

plt.show()

In [ ]:
df["note"].to_numpy()


In [ ]:
notes = df['note'].unique()
notes.sort()
word_cloud = pd.DataFrame(columns=notes)
bag_of_words = pd.DataFrame(columns=notes)


In [ ]:
for comment, note  in df[['commentaire', 'note']].to_numpy()  :
    print(note)
    break

In [ ]:
for comment, note in tqdm(df[['commentaire', 'note']].to_numpy()) :
    if comment and note :
        nlp_comment = nlp(comment)
        for token in nlp_comment :
            if not token.is_stop :
                if not token.lemma_ in bag_of_words.index :
                    bag_of_words.loc[token.lemma_] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                bag_of_words.loc[token.lemma_][note] += 1

In [ ]:
bag_of_words.to_csv('out.csv')
bag_of_words

In [ ]:
word_cloud.loc["text"] = ["", "", "", "", "", "", "", "", "", ""]
for comment, note in tqdm(df[['commentaire', 'note']].to_numpy()) :
    if comment and note :
        nlp_comment = nlp(comment)
        for token in nlp_comment :
            if not token.is_stop :
                word_cloud.loc["text"].loc[note] += token + ' '

In [ ]:
note = "0,5"

# Read the text.
# text = open(word_cloud.loc["text"].loc[note]).read()
text = word_cloud.loc["text"].loc[note]

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()